table of contents
1. データの読み込み
1. 前処理
1. 可視化

In [1]:
import os

import pandas as pd

In [2]:
data_dir = os.path.join('..', 'data')

# データの読み込み

## 新卒マスタ

In [3]:
mst_df = pd.read_csv(os.path.join(data_dir, 'shinsotsu-player.csv'))

In [4]:
mst_df.head()

,player_name,from_team_name,年度,competition
0,茂木駿佑,仙台ユース,2015年,Ｊ１リーグ
1,西村拓真,富山第一高,2015年,Ｊ１リーグ
2,摂津颯登,山形ユース,2015年,Ｊ１リーグ
3,高木利弥,神奈川大,2015年,Ｊ１リーグ
4,大橋尚志,鹿島ユース,2015年,Ｊ１リーグ


In [5]:
pivoted_df = pd.pivot(mst_df.groupby(['年度', 'competition'], as_index=False).agg(n_player_total=('player_name', 'nunique')), index='年度', columns='competition', values='n_player_total')

pivoted_df.style.background_gradient(cmap='Greens', vmin=pivoted_df.min().min(), vmax=pivoted_df.max().max())

competition,Ｊ１リーグ,Ｊ２リーグ,Ｊ３リーグ
年度,,,
2015年,44,67,42
2016年,54,40,44
2017年,48,58,44
2018年,57,61,28
2019年,48,72,50
2020年,64,70,64


## 出場時間データ

In [7]:
df_list = list()
for year in mst_df['年度'].unique():
    if year in ['2015年', '2016年']:
        continue
    competiiton = 'Ｊ１リーグ'
    data_dir_tmp = os.path.join(data_dir, year, competiiton)
    mst_df_tmp = mst_df[(mst_df['年度']==year)&(mst_df['competition']==competiiton)]
    for infile in os.listdir(data_dir_tmp):
        df = pd.read_csv(os.path.join(data_dir_tmp, infile), index_col=0)
        unpivoted_df = pd.melt(df[df.index.isin(mst_df_tmp.player_name.tolist())], ignore_index=False).reset_index()
        unpivoted_df.columns = ['player_name', '節', 'value']
        unpivoted_df['to_team_name'] = infile.split('.')[0]
        df_tmp = pd.merge(unpivoted_df, mst_df_tmp, on='player_name', how='left')
        df_list.append(df_tmp)
        
df = pd.concat(df_list)

# 前処理

In [8]:
mst_df_tmp = mst_df[mst_df['competition']==competiiton]
merged_df = pd.merge(mst_df_tmp[mst_df_tmp['年度'].isin(['2017年', '2018年', '2019年', '2020年'])], df[['年度', 'player_name', 'to_team_name']].drop_duplicates(), on=['年度', 'player_name'], how='left', indicator=True)

agg_df = pd.merge(
        merged_df[merged_df._merge=='left_only'].groupby('年度', as_index=False).agg(n_player_zero=('player_name', 'nunique'))
        , pd.merge(
            df.groupby('年度', as_index=False).agg(n_player_not_zero=('player_name', 'nunique'))
            , mst_df_tmp.groupby('年度', as_index=False).agg(n_player_total=('player_name', 'nunique'))
            , on='年度'
        )
        , on='年度'
    )

agg_df['percentage'] = agg_df.n_player_not_zero / agg_df.n_player_total * 100

In [9]:
agg_df.set_index('年度').iloc[:, 1:].style.background_gradient(cmap='Greens', subset='percentage')

,n_player_not_zero,n_player_total,percentage
年度,,,
2017年,24,48,50.000000
2018年,34,57,59.649123
2019年,35,48,72.916667
2020年,45,64,70.312500
